<img src="./static/imo_health.png" alt="IMO Health Logo" width="300"/>
 
---
# Medallion Data Lake Architecture: Patient Data Pipeline

This notebook demonstrates a Medallion architecture for patient data using Amazon S3 and IMO Health APIs.

## Architecture Overview

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                      MEDALLION DATA LAKE ARCHITECTURE                        │
└─────────────────────────────────────────────────────────────────────────────┘

    ┌──────────────┐         ┌──────────────┐         ┌──────────────┐
    │              │         │              │         │              │
    │   🥉 BRONZE  │────────▶│  🥈 SILVER   │────────▶│   🥇 GOLD    │
    │     ZONE     │         │     ZONE     │         │     ZONE     │
    │              │         │              │         │              │
    └──────────────┘         └──────────────┘         └──────────────┘
         │                          │                        │
         │                          │                        │
    Raw Patient              Normalized Data          Cohort-Filtered
    Data Ingestion           (IMO Health Precision     Patient Records
    (CSV files)              Normalize API)           (Final Analytics)
         │                          │                        │
         │                          │                        │
    • Unprocessed          • Standardized codes       • Business-ready
    • As-is storage        • LOINC, ICD10CM           • Criteria-matched
    • Historical           • RXNORM, CPT              • Decision support
      archive              • Quality improved         • Reporting
```

## Pipeline Steps:

1. Import required libraries
2. Connect to Data Lake Storage (S3)
3. Upload files to the Bronze zone
4. Read data from the Bronze zone
5. Normalize data using IMO Precision Normalize API (Silver zone)
6. Apply cohort criteria and write results to the Gold zone


## 1. Import Required Libraries

Import libraries such as os, pandas, and boto3 for connecting to Amazon S3.

In [ ]:
# Install required packages if not already installed
!pip install pandas boto3 requests ipykernel python-dotenv

import pandas as pd
import boto3
import requests

## 1a. Set (or create .env file) & Load Environment Variables from .env File

Use python-dotenv to load environment variables from a .env file for secure and convenient configuration.

In [ ]:
import os
from dotenv import load_dotenv
os.environ["AWS_ACCESS_KEY_ID"]= "Replace"
os.environ["AWS_SECRET_ACCESS_KEY"]= "Replace"
os.environ["AWS_SESSION_TOKEN"]= "Replace"
os.environ["IMO_CLIENT_ID"]= "Replace"
os.environ["IMO_CLIENT_SECRET"]= "Replace"
load_dotenv()


## 2. Connect to Data Lake Storage

Establish a connection to Amazon S3 using boto3.

In [ ]:
# Connect to S3 using default credential provider chain (recommended)
import boto3
import os
s3 = boto3.client(
    's3',
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
    aws_session_token=os.getenv('AWS_SESSION_TOKEN')  # Optional
)
bronze_bucket = 'your-bronze-bucket'
silver_bucket = 'your-silver-bucket'
gold_bucket = 'your-gold-bucket'

## 3. Get IMO Health API Auth Token

Obtain an authentication token from the IMO OAuth endpoint before making API requests.

In [ ]:
import requests
import os

# Get IMO API Auth Token
auth_url = "https://auth.imohealth.com/oauth/token"
client_id = os.getenv('IMO_CLIENT_ID')
client_secret = os.getenv('IMO_CLIENT_SECRET')
print(client_id)
payload = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
    'audience': 'https://api.imohealth.com'
}
response = requests.post(auth_url, data=payload)
if response.status_code == 200:
    access_token = response.json().get('access_token')
    print("Successfully obtained access token.")
else:
    print(f"Failed to obtain access token: {response.text}")

## 4. Upload Files in Data Lake

List the files and directories available in the Bronze S3 bucket.

In [ ]:
import os
import boto3
from botocore.exceptions import NoCredentialsError, PartialCredentialsError

# Upload a CSV file to the Bronze S3 bucket
print(f"Current working directory: {os.getcwd()}")
local_file = './DQA_Input.csv'  # Path to your local CSV file
bronze_key = 'bronze/patient_data.csv'  # S3 key for the file
try:
    s3.upload_file(local_file, bronze_bucket, bronze_key)
    print(f'Uploaded {local_file} to s3://{bronze_bucket}/{bronze_key}')
except (NoCredentialsError, PartialCredentialsError):
    print("ERROR: AWS credentials not found. Please configure your credentials using AWS CLI or environment variables.")
except FileNotFoundError:
    print(f"ERROR: File {local_file} not found. Please check the path.")
except Exception as e:
    print(f"ERROR: {e}")

# List files in the Bronze bucket
try:
    response = s3.list_objects_v2(Bucket=bronze_bucket, Prefix='bronze/')
    for obj in response.get('Contents', []):
        print(obj['Key'])
except (NoCredentialsError, PartialCredentialsError):
    print("ERROR: AWS credentials not found. Please configure your credentials using AWS CLI or environment variables.")
except Exception as e:
    print(f"ERROR: {e}")

## 5. Read Data from Data Lake

Read a sample data file from the Bronze S3 bucket into a pandas DataFrame for analysis.

In [ ]:
# Read a sample CSV from Bronze zone
bronze_key = 'bronze/patient_data.csv'
s3.download_file(bronze_bucket, bronze_key, '/tmp/patient_data.csv')
df = pd.read_csv('/tmp/patient_data.csv')
df.head()

## 6. Process and Analyze Data

Normalize the data using IMO Precision Normalize API and write the output to the Silver S3 bucket.

In [ ]:
# Normalize data using IMO Health Precision Normalize API (batch, following NormalizeRunnerFB.py style)
import os
import json
import uuid

# Set up API endpoint and headers (replace with your actual credentials/token)
url = "https://api.imohealth.com/precision/normalize"
headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    'imo-bypass-cache': 'true',
    'imo-origin-client-id': os.getenv('IMO_CLIENT_ID', ''),
    'Authorization': f"Bearer {access_token}"
}

def build_requests_array(df):
    requests_array = []
    for i, row in df.iterrows():
        requests_array.append({
            "record_id": str(row.get('record_id', uuid.uuid4())),
            "domain": row.get('domain', ''),
            "input_term": row.get('input_term', ''),
            "context": {
                "order_name": row.get('order_name', ''),
                "unit_of_measure": row.get('component-uom', ''),
            }
        })
    return requests_array

batch_size = 10
results = []
for start in range(0, len(df), batch_size):
    batch_df = df.iloc[start:start+batch_size]
    requests_array = build_requests_array(batch_df)
    payload = json.dumps({
        "client_request_id": str(uuid.uuid4()),
        "preferences": {
            "threshold": 0,
            "debug": False,
            "use_llm_transformation": False,
            "use_llm_select_candidate": False
        },
        "requests": requests_array
    })
    response = requests.post(url, headers=headers, data=payload)
    response_data = response.json()
    # Attach results to batch_df
    for i, response_item in enumerate(response_data.get('requests', [])):
        row = batch_df.iloc[i].to_dict()
        resp = response_item.get('response', {})
        row['Explanation'] = resp.get('explanation', '')
        row['Transformed Term'] = resp.get('transformed_term', '')
        # Default values
        row['Default Lexical Title'] = ''
        row['Default Lexical Code'] = ''
        row['Lexical Title'] = ''
        row['Lexical Code'] = ''
        row['Score'] = 0
        row['LOINC'] = ''
        row['ICD10CM'] = ''
        row['RXNORM'] = ''
        row['CPT'] = ''
        if resp.get('items') and len(resp['items']) > 0:
            item = resp['items'][0]
            row['Default Lexical Title'] = item.get('default_lexical_title', '')
            row['Default Lexical Code'] = item.get('default_lexical_code', '')
            row['Lexical Title'] = item.get('lexical_title', '')
            row['Lexical Code'] = item.get('lexical_code', '')
            row['Score'] = item.get('score', 0)
            metadata = item.get('metadata', {})
            # Extract codes from metadata['mappings'] if present
            try:
                mappings = metadata.get('mappings', {})
                # LOINC for lab domain
                if row.get('domain', '').lower() == 'lab':
                    loinc = mappings.get('loinc', {})
                    loinc_codes = loinc.get('codes', [])
                    if loinc_codes and isinstance(loinc_codes, list):
                        row['LOINC'] = loinc_codes[0].get('code', '')
                # ICD10CM for problem domain
                if row.get('domain', '').lower() == 'problem':
                    icd10cm = mappings.get('icd10cm', {})
                    icd10cm_codes = icd10cm.get('codes', [])
                    if icd10cm_codes and isinstance(icd10cm_codes, list):
                        row['ICD10CM'] = icd10cm_codes[0].get('code', '')
                # RXNORM for medication domain
                if row.get('domain', '').lower() == 'medication':
                    rxnorm = mappings.get('rxnorm', {})
                    rxnorm_codes = rxnorm.get('codes', [])
                    if rxnorm_codes and isinstance(rxnorm_codes, list):
                        row['RXNORM'] = rxnorm_codes[0].get('code', '')
                # CPT for procedure domain
                if row.get('domain', '').lower() == 'procedure':
                    cpt = mappings.get('cpt', {})
                    cpt_codes = cpt.get('codes', [])
                    if cpt_codes and isinstance(cpt_codes, list):
                        row['CPT'] = cpt_codes[0].get('code', '')
            except Exception as e:
                pass
        results.append(row)

norm_df = pd.DataFrame(results)
norm_df.to_csv('/tmp/normalized.csv', index=False)
s3.upload_file('/tmp/normalized.csv', silver_bucket, 'silver/normalized_patient_data.csv')
print('Normalized data written to Silver zone. LOINC, ICD10CM, RXNORM, CPT columns extracted from metadata.mappings.')

## 6. Preview Normalized Data in Silver Zone


In [ ]:
# Preview the normalized dataset from the Silver zone before cohorting
s3.download_file(silver_bucket, 'silver/normalized_patient_data.csv', '/tmp/normalized.csv')
norm_df = pd.read_csv('/tmp/normalized.csv')
display(norm_df.head())

## 7. Select and Download Cohort Criteria definition
1.  Select Cohort Criteria defined in  IMO Health Precision Set Platform:

        - https://studio.imohealth.com/#/valuesets/editor/individual/20294/version/66922
        - Scope: Finding all tests related to  anti mitochondria antibody

2. Download the valueset defined



In [ ]:
import pandas as pd
import csv
import os

def search_valuesets(search_term, token):
    """Search for valuesets using the IMO Health API"""
    url = "https://api.imohealth.com/fhir/r6/ValueSet/search"
    
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {token}'
    }
    params = {"searchText": search_term}

    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Failed to search valuesets: {response.status_code} - {response.text}")

def get_valueset_codes_paged(valueset_id, token):
    """Get all codes from a valueset with pagination"""
    all_codes = []
    page = 1
    page_size = 50
    max_pages = 100
    
    while page <= max_pages:
        url = f"https://api.imohealth.com/fhir/r6/ValueSet/{valueset_id}"
        
        headers = {
            'Authorization': f'Bearer {token}'
        }
        
        params = {
            'page': page,
            'pageSize': page_size
        }
        
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code != 200:
            raise Exception(f"Failed to get valueset codes: {response.status_code} - {response.text}")
        
        data = response.json()
        expansion = data.get('expansion', {})
        total = expansion.get('total', 0)   
        codes = expansion.get('contains', [])

        if not codes:
            break
            
        all_codes.extend(codes)
        print(f"Retrieved page {page}, codes on this page: {len(codes)}, total codes so far: {len(all_codes)}")
        
        if len(all_codes) >= total or len(codes) < page_size:
            print(f"Pagination complete. Total codes retrieved: {len(all_codes)}")
            break
            
        page += 1
    
    if page > max_pages:
        print(f"⚠️ Reached maximum page limit ({max_pages}). Retrieved {len(all_codes)} codes.")
    
    return all_codes

def save_codes_to_csv(codes, filename):
    """Save codes to CSV file"""
    filepath = os.path.join('.', filename)
    
    with open(filepath, 'w', newline='', encoding='utf-8') as csvfile:
        if codes:
            fieldnames = codes[0].keys()
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(codes)
    
    return filepath

# Download precision set for anti mitochondria antibody tests
valueset_id = "20294"  # ID from the precision set URL
print(f"📥 Downloading precision set (ID: {valueset_id}) for anti mitochondria antibody tests...")

try:
    codes = get_valueset_codes_paged(valueset_id, access_token)
    
    if codes:
        filename = f"precision_set_{valueset_id}_codes.csv"
        filepath = save_codes_to_csv(codes, filename)
        
        print(f"✅ Successfully downloaded {len(codes)} codes!")
        print(f"📁 File saved as: {filename}")
        
        # Load into DataFrame for cohort matching
        valueset_df = pd.DataFrame(codes)
        print(f"\n📊 Valueset preview:")
        display(valueset_df.head())
        
        # Extract codes into a set for matching
        valueset_codes = set(valueset_df['code'].values)
        print(f"\n✅ Loaded {len(valueset_codes)} unique codes for cohort matching")
        
    else:
        print("⚠️ No codes found for this valueset.")
        
except Exception as e:
    print(f"❌ Error downloading precision set: {str(e)}")

## Apply Cohorting Rules based on the downloaded precision set and copy the finalized output to the Gold Zone


In [ ]:
## Apply cohorting rules based on the downloaded precision set
# Filter normalized DataFrame for records matching the valueset codes
cohort_df = norm_df[norm_df['LOINC'].isin(valueset_codes)]
print(f"Total records in cohort matching precision set: {len(cohort_df)}")
cohort_df.to_csv('/tmp/cohort_data.csv', index=False)

## Move to Gold zone
s3.upload_file('/tmp/cohort_data.csv', gold_bucket, 'gold/cohort_patient_data.csv')
print('Cohort data written to Gold zone.')

## 8. Preview Final Cohort Data in Gold Zone

Display the final cohorted patient data that was written to the Gold zone.

In [ ]:
# Preview the final cohort data from the Gold zone
s3.download_file(gold_bucket, 'gold/cohort_patient_data.csv', '/tmp/cohort_data.csv')
gold_df = pd.read_csv('/tmp/cohort_data.csv')
print(f"📊 Total records in Gold zone: {len(gold_df)}")
print(f"\n🎯 Final cohort data preview:")
display(gold_df.head(10))

## Summary

This notebook demonstrated a Medallion Data Lake pipeline for patient data using S3 and IMO APIs:

- Raw data ingested to Bronze zone
- Normalized to Silver zone using IMO Precision Normalize API
- Cohort criteria applied and results moved to Gold zone